In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from os import listdir
from os.path import isfile, join
import numpy as np
import tensorflow as tf
from PIL import Image as img
from tensorflow.python import keras
from keras import Sequential, datasets, layers, models
from keras.preprocessing.image import ImageDataGenerator
import pickle as pk

In [ ]:
pixels = 150
dimensions = (pixels, pixels)
bins = 4
mid_layer = 128
num_epochs = 3
learning_rate = 0.001
reduction_factor = 0.8

In [ ]:
class LearningRateReducerCb(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        old_lr = self.model.optimizer.lr.read_value()
        new_lr = old_lr * reduction_factor
        print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

In [ ]:
Generator = ImageDataGenerator(rescale=1./255, validation_split=0.15)

target_labels = [1, 2, 0, 3]

folder_path = "/mnt/c/NN/Brain_Tumor/Training"
train_gen = Generator.flow_from_directory(folder_path,
                                            target_size=dimensions,
                                            color_mode='rgb',
                                            classes=target_labels,
                                            class_mode='categorical',
                                            batch_size=32,
                                            shuffle=True,
                                            subset='training')

val_gen = Generator.flow_from_directory(folder_path,
                                            target_size=dimensions,
                                            color_mode='rgb',
                                            classes=target_labels,
                                            class_mode='categorical',
                                            batch_size=32,
                                            shuffle=True,
                                            subset='validation')

In [ ]:
train_images = []
train_labels = np.array([])

folder_path = "/mnt/c/NN/Brain_Tumor/Training/"

for i in range(10, 1321):
    im = img.open(folder_path + "glioma/Tr-gl_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 1)

for i in range(10, 1339):
    im = img.open(folder_path + "meningioma/Tr-me_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 2)

for i in range(10, 1457):
    im = img.open(folder_path + "pituitary/Tr-pi_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 3)

for i in range(10, 1595):
    im = img.open(folder_path + "notumor/Tr-no_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im_array = np.asarray(im)
    im.close()
    train_images.append(im_array)
    train_labels = np.append(train_labels, 0)

In [ ]:
test_images = []
test_labels = np.array([])

folder_path = "/mnt/c/NN/Brain_Tumor/Testing/"

for i in range(10, 300):
    im = img.open(folder_path + "glioma/Te-gl_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im_array = np.asarray(im)
    im.close()
    test_images.append(im_array)
    test_labels = np.append(test_labels, 1)

for i in range(10, 306):
    im = img.open(folder_path + "meningioma/Te-me_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im_array = np.asarray(im)
    im.close()
    test_images.append(im_array)
    test_labels = np.append(test_labels, 2)

for i in range(10, 300):
    im = img.open(folder_path + "pituitary/Te-pi_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im_array = np.asarray(im)
    im.close()
    test_images.append(im_array)
    test_labels = np.append(test_labels, 3)

for i in range(10, 405):
    im = img.open(folder_path + "notumor/Te-no_" + str(i).zfill(4) + ".jpg")
    im = im.resize(dimensions)
    im_array = np.asarray(im)
    im.close()
    test_images.append(im_array)
    test_labels = np.append(test_labels, 0)

In [ ]:
temp = np.array([])
for i in range(len(train_images)):
    temp = np.append(temp, train_images[i])
train_images = temp
print(np.shape(train_images))

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(pixels, (3, 3), activation='relu', input_shape=(pixels, pixels, 1)))
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2 * pixels, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(bins))

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, callbacks=[LearningRateReducerCb()], epochs=num_epochs,
                    validation_data=(test_images, test_labels))

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Accuracy = " + str(np.round(test_acc * 100, 3)) + " %")

In [ ]:
filename = 'model.bin'
pk.dump(model, open(filename, 'wb'))

In [ ]:
model = pk.load(open(filename, 'rb'))